In [1]:
# http://rasbt.github.io/mlxtend
# https://programmerall.com/article/9818116902/  -- The 3 algorithms in the same library

In [2]:
# Install mlxtend (optional)
#!pip uninstall --yes mlxtend
#!pip install mlxtend

In [3]:
# Generic imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Imports to handle transaction datasets
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

# Dataset

In [5]:
# Sample dataset
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]
dataset

[['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
 ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
 ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
 ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
 ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

In [6]:
# Format data into binary matrix
trans_encoder = TransactionEncoder() # Create an instance of a TransactionEncoder
trans_encoder.fit(dataset)           # Teach the transaction encoder the list of all possible items
dataset_binary = trans_encoder.transform(dataset) # Map the dataset into binary form

print("There are", dataset_binary.shape[0], "transactions, and", dataset_binary.shape[1], "possible items.")
dataset_binary

There are 5 transactions, and 11 possible items.


array([[False, False, False,  True, False,  True,  True,  True,  True,
        False,  True],
       [False, False,  True,  True, False,  True, False,  True,  True,
        False,  True],
       [ True, False, False,  True, False,  True,  True, False, False,
        False, False],
       [False,  True, False, False, False,  True,  True, False, False,
         True,  True],
       [False,  True, False,  True,  True,  True, False, False,  True,
        False, False]])

In [9]:
# Put the binary matrix into a data frame
dataset_df = pd.DataFrame(dataset_binary, columns=trans_encoder.columns_, dtype=bool)
dataset_df

Apple   Corn   Dill   Eggs  Ice cream  Kidney Beans   Milk  Nutmeg  Onion  \
0  False  False  False   True      False          True   True    True   True   
1  False  False   True   True      False          True  False    True   True   
2   True  False  False   True      False          True   True   False  False   
3  False   True  False  False      False          True   True   False  False   
4  False   True  False   True       True          True  False   False   True   

   Unicorn  Yogurt  
0    False    True  
1    False    True  
2    False   False  
3     True    True  
4    False   False

# Apriori

In [10]:
# Get the list of frequent itemsets with at least 60% support
itemsets = apriori(dataset_df, min_support=0.6)
itemsets

support   itemsets
0       0.8        (3)
1       1.0        (5)
2       0.6        (6)
3       0.6        (8)
4       0.6       (10)
5       0.8     (3, 5)
6       0.6     (8, 3)
7       0.6     (5, 6)
8       0.6     (8, 5)
9       0.6    (10, 5)
10      0.6  (8, 3, 5)

In [11]:
# Now, get it again but with the item names insted of their indices
itemsets = apriori(dataset_df, min_support=0.6, use_colnames=True)
itemsets

support                     itemsets
0       0.8                       (Eggs)
1       1.0               (Kidney Beans)
2       0.6                       (Milk)
3       0.6                      (Onion)
4       0.6                     (Yogurt)
5       0.8         (Kidney Beans, Eggs)
6       0.6                (Onion, Eggs)
7       0.6         (Kidney Beans, Milk)
8       0.6        (Onion, Kidney Beans)
9       0.6       (Kidney Beans, Yogurt)
10      0.6  (Onion, Kidney Beans, Eggs)

In [12]:
# Create a new col with the length of each itemset
itemsets['length'] = itemsets['itemsets'].apply(lambda x: len(x))
itemsets

support                     itemsets  length
0       0.8                       (Eggs)       1
1       1.0               (Kidney Beans)       1
2       0.6                       (Milk)       1
3       0.6                      (Onion)       1
4       0.6                     (Yogurt)       1
5       0.8         (Kidney Beans, Eggs)       2
6       0.6                (Onion, Eggs)       2
7       0.6         (Kidney Beans, Milk)       2
8       0.6        (Onion, Kidney Beans)       2
9       0.6       (Kidney Beans, Yogurt)       2
10      0.6  (Onion, Kidney Beans, Eggs)       3

In [13]:
# Selection of a subset of itemsets according to arbitrary parameters
itemsets[(itemsets['length'] == 2) & (itemsets['support'] >= 0.8)]

support              itemsets  length
5      0.8  (Kidney Beans, Eggs)       2

In [14]:
# Now select a subset containing especific itemsets
itemsets[itemsets['itemsets'] == {'Onion', 'Eggs'}]

support       itemsets  length
6      0.6  (Onion, Eggs)       2

In [15]:
# From the previous set of frequent itemsets, get its association rules with confidence >= 0.7
rules = association_rules(itemsets, metric="confidence", min_threshold=0.7)
rules.sort_values(by=['lift'], ascending=False)

antecedents            consequents  antecedent support  \
2                 (Onion)                 (Eggs)                 0.6   
7   (Onion, Kidney Beans)                 (Eggs)                 0.6   
10                (Onion)   (Kidney Beans, Eggs)                 0.6   
3                  (Eggs)                (Onion)                 0.8   
9    (Kidney Beans, Eggs)                (Onion)                 0.8   
11                 (Eggs)  (Onion, Kidney Beans)                 0.8   
0          (Kidney Beans)                 (Eggs)                 1.0   
1                  (Eggs)         (Kidney Beans)                 0.8   
4                  (Milk)         (Kidney Beans)                 0.6   
5                 (Onion)         (Kidney Beans)                 0.6   
6                (Yogurt)         (Kidney Beans)                 0.6   
8           (Onion, Eggs)         (Kidney Beans)                 0.6   

    consequent support  support  confidence  lift  leverage  conviction  
2                  0.8      0.6        1.00  1.25      0.12         inf  
7                  0.8      0.6        1.00  1.25      0.12         inf  
10                 0.8      0.6        1.00  1.25      0.12         inf  
3                  0.6      0.6        0.75  1.25      0.12         1.6  
9                  0.6      0.6        0.75  1.25      0.12         1.6  
11                 0.6      0.6        0.75  1.25      0.12         1.6  
0                  0.8      0.8        0.80  1.00      0.00         1.0  
1                  1.0      0.8        1.00  1.00      0.00         inf  
4                  1.0      0.6        1.00  1.00      0.00         inf  
5                  1.0      0.6        1.00  1.00      0.00         inf  
6                  1.0      0.6        1.00  1.00      0.00         inf  
8                  1.0      0.6        1.00  1.00      0.00         inf

In [16]:
# Now get those rules with lift >= 1.2
rules = association_rules(itemsets, metric="lift", min_threshold=1.2)
rules

antecedents            consequents  antecedent support  \
0                (Onion)                 (Eggs)                 0.6   
1                 (Eggs)                (Onion)                 0.8   
2  (Onion, Kidney Beans)                 (Eggs)                 0.6   
3   (Kidney Beans, Eggs)                (Onion)                 0.8   
4                (Onion)   (Kidney Beans, Eggs)                 0.6   
5                 (Eggs)  (Onion, Kidney Beans)                 0.8   

   consequent support  support  confidence  lift  leverage  conviction  
0                 0.8      0.6        1.00  1.25      0.12         inf  
1                 0.6      0.6        0.75  1.25      0.12         1.6  
2                 0.8      0.6        1.00  1.25      0.12         inf  
3                 0.6      0.6        0.75  1.25      0.12         1.6  
4                 0.8      0.6        1.00  1.25      0.12         inf  
5                 0.6      0.6        0.75  1.25      0.12         1.6

In [17]:
# Use a lambda function to compute the length of each antecedent
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

antecedents            consequents  antecedent support  \
0                (Onion)                 (Eggs)                 0.6   
1                 (Eggs)                (Onion)                 0.8   
2  (Onion, Kidney Beans)                 (Eggs)                 0.6   
3   (Kidney Beans, Eggs)                (Onion)                 0.8   
4                (Onion)   (Kidney Beans, Eggs)                 0.6   
5                 (Eggs)  (Onion, Kidney Beans)                 0.8   

   consequent support  support  confidence  lift  leverage  conviction  \
0                 0.8      0.6        1.00  1.25      0.12         inf   
1                 0.6      0.6        0.75  1.25      0.12         1.6   
2                 0.8      0.6        1.00  1.25      0.12         inf   
3                 0.6      0.6        0.75  1.25      0.12         1.6   
4                 0.8      0.6        1.00  1.25      0.12         inf   
5                 0.6      0.6        0.75  1.25      0.12         1.6   

   antecedent_len  
0               1  
1               1  
2               2  
3               2  
4               1  
5               1

In [18]:
# Show only association rules with specific length and confidence
rules[(rules['antecedent_len'] >= 2) & (rules['confidence'] > 0.75) & (rules['lift'] > 1.2)]

antecedents consequents  antecedent support  consequent support  \
2  (Onion, Kidney Beans)      (Eggs)                 0.6                 0.8   

   support  confidence  lift  leverage  conviction  antecedent_len  
2      0.6         1.0  1.25      0.12         inf               2

# FPgrowth

In [19]:
# Get the list of frequent itemsets with at least 60% support
itemsets = fpgrowth(dataset_df, min_support=0.6, use_colnames=True)
itemsets

support                     itemsets
0       1.0               (Kidney Beans)
1       0.8                       (Eggs)
2       0.6                     (Yogurt)
3       0.6                      (Onion)
4       0.6                       (Milk)
5       0.8         (Kidney Beans, Eggs)
6       0.6       (Kidney Beans, Yogurt)
7       0.6                (Onion, Eggs)
8       0.6        (Onion, Kidney Beans)
9       0.6  (Onion, Kidney Beans, Eggs)
10      0.6         (Kidney Beans, Milk)

In [20]:
# From the previous set of frequent itemsets, get its association rules with confidence >= 0.7
rules = association_rules(itemsets, metric="confidence", min_threshold=0.7)
rules.sort_values(by=['lift'], ascending=False)

antecedents            consequents  antecedent support  \
3                 (Onion)                 (Eggs)                 0.6   
6   (Onion, Kidney Beans)                 (Eggs)                 0.6   
9                 (Onion)   (Kidney Beans, Eggs)                 0.6   
4                  (Eggs)                (Onion)                 0.8   
8    (Kidney Beans, Eggs)                (Onion)                 0.8   
10                 (Eggs)  (Onion, Kidney Beans)                 0.8   
0          (Kidney Beans)                 (Eggs)                 1.0   
1                  (Eggs)         (Kidney Beans)                 0.8   
2                (Yogurt)         (Kidney Beans)                 0.6   
5                 (Onion)         (Kidney Beans)                 0.6   
7           (Onion, Eggs)         (Kidney Beans)                 0.6   
11                 (Milk)         (Kidney Beans)                 0.6   

    consequent support  support  confidence  lift  leverage  conviction  
3                  0.8      0.6        1.00  1.25      0.12         inf  
6                  0.8      0.6        1.00  1.25      0.12         inf  
9                  0.8      0.6        1.00  1.25      0.12         inf  
4                  0.6      0.6        0.75  1.25      0.12         1.6  
8                  0.6      0.6        0.75  1.25      0.12         1.6  
10                 0.6      0.6        0.75  1.25      0.12         1.6  
0                  0.8      0.8        0.80  1.00      0.00         1.0  
1                  1.0      0.8        1.00  1.00      0.00         inf  
2                  1.0      0.6        1.00  1.00      0.00         inf  
5                  1.0      0.6        1.00  1.00      0.00         inf  
7                  1.0      0.6        1.00  1.00      0.00         inf  
11                 1.0      0.6        1.00  1.00      0.00         inf